# *MEEB* Run Script Tutorial - Deterministic Version

Notebook tutorial for running the Mesoscale Explicit Ecogeomorphic Barrier model (*MEEB*) v1.1 deterministically. After exploring this notebook first, see the notebook `run_MEEB_Probabilistic.ipynb` for a tutorial on running MEEB *probabilistically*.

For general model information and installation instructions, see the `README` in the main project directory.

To run *MEEB* deterministically outside of this notebook, execute the `run_MEEB.py` script located in the `/Tools` folder, upon which the code in this tutorial is based. The run script should be executed from the main directory to access required inputs.

Last updated: 26 June 2025

## Variables and Initializations

First, import model requirements. This includes model functions stored in the `routines_meeb.py` file and the model class `MEEB` from the main `meeb.py` file. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from tqdm import trange
import os

os.chdir(os.path.pardir)  # Set the working directory to the main MEEB folder (the parent directory of the Notebooks subdirectory)

import routines_meeb as routine 
from meeb import MEEB

Check that the current working directory is the main MEEB directory (not its Notebooks subdirectory where the notebook is stored).

In [ ]:
print(os.getcwd())

Next, create an instance of the MEEB class and define model input parameter values. All inputs parameters not defined here (there are many more than given below!) will follow their *default* values/specifications defined in the `meeb.py` file.

For this tutorial, we will run a 5-yr-long deterministic forecast for a 500 m section of North Core Banks, NC, USA, beginning October 2018 and ending October 2023. You can experiment by change the parameters values below. For example changing the alongshore domain boundary min and max will change the location on North Core Banks and/or size of the model domain. The initial elevation and vegetation file used in this tutorial covers onlt a 5 km portion of North Core Banks; the init file that covers the entirety of North Core Banks is too large for storing in this repository.

In [ ]:
# Create an instance of the MEEB class
meeb = MEEB(
    name="MEEB Deterministic Run Script Tutorial",
    simulation_time_yr=5,  # [yr] Duration of simulation
    cellsize=2,  # [m] Horizontal cell dimensions
    alongshore_domain_boundary_min=250,  # [cellsize] Alongshore minimum coordinate
    alongshore_domain_boundary_max=750,  # [cellsize] Alongshore maximum coordinate
    crossshore_domain_boundary_min=100,  # [cellsize] Cross-shore minimum coordinate
    crossshore_domain_boundary_max=850,  # [cellsize] Cross-shore maximum coordinate
    RSLR=0.0124,  # [m/yr] Relative sea-level rise rate
    init_filename="Init_NCB-NewDrum-Ocracoke_2018_PostFlorence_18400-23400.npy",  # Initial elevation and vegetation file, stored in the Input folder
    simulation_start_date="20181007",  # [yyyymmdd] Date at which to begin the simulation; should be date of initial elevation/vegetation capture
    seeded_random_numbers=True,  # [bool] Determines whether to use seeded random number generator for reproducibility
)

## Run the Model

To run the model, loop through each timestep using the `update()` function. A progress bar that prints to screen as the model runs is also added. It will likely take 2-5 minutes for this simulation to complete.

In [ ]:
# Loop through time
with trange(int(meeb.iterations)) as t:
    for time_step in t:
        # Run time step
        meeb.update(time_step)
        # Update progress bar
        t.set_postfix({'Year': "{:.2f}".format((time_step + 1) / meeb.iterations_per_cycle) + '/' + "{:.2f}".format(meeb.simulation_time_yr)})
        t.update()

# Without the progress bar, the code above can be simplified to:
# for time_step in range(int(meeb.iterations)):
#     meeb.update(time_step)

## Explore Model Results

Once the model finishes, we can plot results by accessing data components of the `meeb` object. 

### Final Elevation and Vegetation
For starters, we can plot the topography and vegetation at the end of the simulation.

In [ ]:
# Get final topography and vegetation cover
topo_end = meeb.topo_TS[:, :, -1]  # [m NAVDD88] Final recorded topography of simulation
veg_end = meeb.spec1_TS[:, :, -1] + meeb.spec2_TS[:, :, -1]  # [0-1] Final recorded vegetation density of simulation

# Set up colormaps (with masking)
cmap1 = routine.truncate_colormap(copy.copy(plt.colormaps["terrain"]), 0.5, 0.9)  # For topography, modifies the built-in "terrain" cmap
cmap1.set_bad(color='dodgerblue', alpha=0.5)  # Set cells below MHW to blue

cmap2 = copy.copy(plt.colormaps["YlGn"])  # For vegetation
cmap2.set_bad(color='dodgerblue', alpha=0.5)  # Set cells below MHW to blue

# Plot
Fig = plt.figure(figsize=(14, 7.5))
Fig.suptitle(meeb.name, fontsize=13)

topo_end = np.ma.masked_where(topo_end <= meeb.MHW, topo_end)  # Mask cells below MHW
ax1 = Fig.add_subplot(211)
cax1 = ax1.matshow(topo_end, cmap=cmap1, vmin=-1, vmax=6.0)  # Plot data with topo colormap
cbar = Fig.colorbar(cax1)  # Add colorbar
cbar.set_label('Elevation [m]', rotation=270, labelpad=20)

ax2 = Fig.add_subplot(212)
veg_end = np.ma.masked_where(topo_end <= meeb.MHW, veg_end)  # Mask cells below MHW
cax2 = ax2.matshow(veg_end, cmap=cmap2, vmin=0, vmax=1)  # Plot data with veg colormap
cbar = Fig.colorbar(cax2)  # Add colorbar
cbar.set_label('Vegetation [%]', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

### Change in Elevation and Vegetation
We can also examine the change in elevation and vegetation by subtracting the topo and veg domains at the last time step from the topo and veg domains at initial time step.

In [ ]:
# Get initial topography and vegetation cover
topo_start = meeb.topo_TS[:, :, 0]  # [m NAVDD88] Final recorded topography of simulation
veg_start = meeb.spec1_TS[:, :, 0] + meeb.spec2_TS[:, :, 0]  # [0-1] Final recorded vegetation density of simulation

# Find change from simulation start to end
topo_change = topo_end - topo_start  # [m]
veg_change = veg_end - veg_start  # [0-1]

# Find maximum elevation change (erosion or accretion)
cmap_lim = max(abs(np.min(topo_change)), abs(np.max(topo_change)))

# Set up colormaps (with masking)
cmap3 = copy.copy(plt.colormaps["bwr_r"])
cmap3.set_bad(color='dodgerblue', alpha=0.5)  # Set cell color below MHW to blue

cmap4 = copy.copy(plt.colormaps["BrBG"])
cmap4.set_bad(color='dodgerblue', alpha=0.5)  # Set cell color below MHW to blue

# Plot changes
Fig = plt.figure(figsize=(14, 7.5))
Fig.suptitle(meeb.name, fontsize=13)

ax1 = Fig.add_subplot(211)
topo_change = np.ma.masked_where(topo_end <= meeb.MHW, topo_change)  # Mask cells below MHW
cax1 = ax1.matshow(topo_change, cmap=cmap3, vmin=-cmap_lim, vmax=cmap_lim)
cbar = Fig.colorbar(cax1)  # Add colorbar
cbar.set_label('Elevation Change [m]', rotation=270, labelpad=20)

ax2 = Fig.add_subplot(212)
veg_change = np.ma.masked_where(topo_end <= meeb.MHW, veg_change)  # Mask cells below MHW
cax2 = ax2.matshow(veg_change, cmap=cmap4, vmin=-1, vmax=1)
cbar = Fig.colorbar(cax2)  # Add colorbar
cbar.set_label('Vegetation Density Change', rotation=270, labelpad=20)

plt.tight_layout()

### Stochastic Storm Record
As another example, we can plot the total water level (TWL) timeseries for storms that stochastically occured in this simulation. As evident in the figure below, the largest storm event (in terms of TWL) occured just after 4 years into the simulation with a TWL of approximately 3.8 m NAVD88.

In [ ]:
Fig = plt.figure(figsize=(10, 5))
storms = meeb.StormRecord
twl_it = (((storms[1:, 0] - 1) * meeb.iterations_per_cycle) + storms[1:, 1]) / meeb.iterations_per_cycle
plt.scatter(twl_it, storms[1:, 2])
plt.xlabel("Simulation Years")
plt.ylabel("TWL (m NAVD88)")

Additional plotting examples (including animations across time!) are given in the `run_MEEB.py` runfile script located in the `/Tools` folder. Animations will save in the `/Output/Animation` directory (be sure to create this directory path if it does not already exist).

## Want to Learn More?

You should hopefully now have a good idea how to run a single deterministic *MEEB* simulation. You can explore the effects different parameter values or settings by altering the parameters values/settings in the instantiation of the `MEEB` class (box 3) or adding parameters not included in this tutorial see additional parameter in the `meeb.py` file. For example, try adding saltation_length (in units of cells) and set its value to 3 (its default value was 5), re-run the model, and see how it changes things.

`MEEB` can also be run to generate __*probabilistic*__ projections of future change. For a tutorial on running *MEEB* probabilistically, see the `run_MEEB_Probabilistic.ipynb` notebook located in the `Notebooks` folder.

MEEB is described in detail in the paper: *Reeves, I. R. B., Ashton, A. D., Lentz, E. L., Sherwood, C. R., Passeri, D. L., & Zeigler., S. L. (2025). Projecting management-relevant change of undeveloped coastal barriers with the Mesoscale Explicit Ecogeomorphic Barrier model (MEEB) v1.0: Geoscientific Model Development Discussions (preprint), https://doi.org/10.5194%2Fgmd-2024-232.* 